## Лаба 8. Рекомендательная система фильмов MovieLens — 1

### Дедлайн

⏰ Четверг, 06 июня 2019 года, 23:59.

### Задача

По имеющимся данным о рейтингах фильмов (MovieLens: 100 000 рейтингов) построить коллаборативную рекомендательную систему. Первая часть (эта лаба) — это подсчёт агрегированных статистик и построение User-User Recommender System на основе коллаборативной фильтрации.

#### Обработка данных на вход

Имеются следующие входные данные:
* Таблица `users * films` с рейтингами. Архив с датасетом нужно скачать с сайта GroupLens: http://files.grouplens.org/datasets/movielens/ml-100k.zip. Это тот же датасет, что и был у вас в [Лабе 6](../lab06/lab06.md), поэтому можете просто взять данные оттуда. Файл `u.data` содержит все оценки, а файл `u.item` — список всех фильмов.
* `id` [642] выданного вам пользователя для расчёта — в вашем Личном кабинете на странице [Лабы 8](http://lk.newprolab.com/lab/laba08).

#### Обработка данных на выход

**Часть1. Основные характеристики данных MovieLens:**

1. Найдите количество всех пользователей и количество всех фильмов в данных (общее число пользователей и фильмов в датасете).
2. Сколько пользователь в среднем ставит рейтингов? Подсчитать `количество рейтингов / количество пользователей`. 
   Поле `average_user_ratings`.
3. Сколько фильм в среднем имеет рейтингов?  Подсчитать`количество рейтингов / количество фильмов`. 
   Поле `average_film_ratings`.
4. Найдите процент заполненных ячеек в данных: `количество рейтингов / (количество пользователей * количество фильмов)`. 
   Поле `completeness`.
5. Какой тип CF вы бы использовали на данном датасете? Ответ не оценивается:
* user-user
* item-item

**Часть 2. User-user CF:**

1. Для каждого пользователя найдите его средний рейтинг (сумма рейтингов пользователя/количество рейтингов пользователя). Здесь I<sub>a</sub> — множество фильмов, по которым у пользователя есть рейтинги r<sub>ui</sub>. Здесь и далее |I<sub>a</sub>| обозначает количество элементов в множестве I<sub>a</sub>.

<img width="100px" src="images/laba08_r_a_avg.png">

Для выданного вам пользователя (ID в личном кабинете):

2. Посчитайте меру близости Пирсона выданного вам пользователя со всеми остальными пользователями. Обратите внимание, что корреляция Пирсона считается только на пересечении, то есть вклад вносят только фильмы, оцененные совместно (I<sub>a</sub>, I<sub>u</sub> — множества оцененных пользователями `a` и `u` фильмов). Корреляция с константой (ситуация, когда у пользователя все оценки одинаковые) равна нулю.

Формат: `ID пользователя; корреляция Пирсона`.

<img width="350px" src="images/laba08_pearson.png">

3. Посчитайте поправочный коэффициент для корреляции Пирсона на нехватку данных:

<img width="150px" src="images/laba08_reg_coef.png">

4. Найдите 30 ближайших пользователей-соседей данного пользователя (`pearson_neighbours`), используя поправленную корреляцию Пирсона: 

<img width="500px" src="images/laba08_pearson_reg_coef.png">

Дальше мы будем использовать только этих соседей для прогноза. Заметим, что строя предсказание для некоторого фильма, мы из фиксированных 30 ближайших пользователей-соседей выбираем только тех, у кого есть оценки для данного фильма, то есть используемых пользователей-соседей для прогноза может быть сильно меньше. Можно находить k ближайших пользователей-соседей, у которых есть оценки по интересующему нас фильму, но это вычислительно гораздо сложнее, поэтому целесообразность этого следует проверять на практике (в рамках данной лабораторной, это лишнее усложнение). 

5. Для всех фильмов найдите прогноз оценки по формуле ниже. Здесь *N(a)* — множество пользователей-соседей, *s(a,u)* — мера близости пользователей из предыдущих пунктов, *|s(a,u)|* — модуль меры близости. 

<img width="250px" src="images/laba08_user_user_cf.png">

6. Рекомендуйте пользователю 10 фильмов (`pearson_top10`), которые он еще не оценивал, с самыми высокими прогнозами из предыдущего пункта.

7. Сравните результат с топ-10 фильмами, полученными по нижней границе доверительного интервала Wilson - поле `top10_lower` из Лабы 6S. Найдите количество общих фильмов в этих двух десятках. 

#### Проверка
Результат следует сохранить в файл `lab08.json` в своей домашней директории в следующем формате:
```
{
    "average_film_ratings": 89.2222,
    "average_user_ratings": 156.4444,
    "completeness": 0.093434,
    "pearson_neighbours": [123, 768, 174, 82, 457, 600, 83, 577, 22, 26, 348, 397, 615, 222, 881, 57, 719, 847, 831, 541, 452, 69, 87, 619, 456, 95, 789, 402, 303, 622],
    "pearson_top10": [1137, 1167, 481, 525, 165, 494, 863, 1427, 968, 1282]
}
```

При равенстве соответствующих метрик сортировать необходимо по возрастанию `id`, рассматривать его нужно как число.

Округление метрик — до 4 знаков после запятой.

Для получения `True` по двум последним ключам **точность совпадения должна быть не менее 0.8**.

Проверка осуществляется [автоматическим скриптом](http://lk.newprolab.com/lab/laba08) из Личного кабинета.

## Решение

In [1]:
import pandas as pd
import json

### Load dataset

In [2]:
# Load the main table with users, films, ratings
df = pd.read_csv('../lab06/ml-100k/u.data', sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
user_id      100000 non-null int64
item_id      100000 non-null int64
rating       100000 non-null int64
timestamp    100000 non-null int64
dtypes: int64(4)
memory usage: 3.1 MB


In [4]:
# Let's make a matrix of ratings (users x movies) for further caclulations.
r = df.pivot(index='user_id', columns='item_id', values='rating')
r.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Часть 1. Основные характеристики данных MovieLens:

#### 1. Найдите кол-во всех пользователей и количество всех фильмов в данных (общее число пользователей и фильмов в датасете).

In [5]:
# Number of users
r.index.size

943

In [6]:
# Number of movies
r.columns.size

1682

#### 2. Сколько пользователь в среднем ставит рейтингов? Подсчитать `количество рейтингов / количество пользователей`. 
Поле `average_user_ratings`.

In [7]:
average_user_ratings = (r.count().sum() / r.index.size).round(4)
average_user_ratings

106.0445

#### 3. Сколько фильм в среднем имеет рейтингов?  Подсчитать`количество рейтингов / количество фильмов`. 
   Поле `average_film_ratings`.

In [8]:
average_film_ratings = (r.count().sum() / r.columns.size).round(4)
average_film_ratings

59.453

#### 4. Найдите процент заполненных ячеек в данных: `количество рейтингов / (количество пользователей * количество фильмов)`. 
   Поле `completeness`.

In [9]:
completeness = (r.count().sum() / (r.index.size * r.columns.size)).round(4)
completeness

0.063

#### 5. Какой тип CF вы бы использовали на данном датасете? Ответ не оценивается:
- user-user !!!
- item-item

### Часть 2. User-user CF:

1. Для каждого пользователя найдите его средний рейтинг (сумма рейтингов пользователя/ количество рейтингов пользователя). Здесь I<sub>a</sub> — множество фильмов, по которым у пользователя есть рейтинги r<sub>ui</sub>. Здесь и далее |I<sub>a</sub>| обозначает количество элементов в множестве I<sub>a</sub>.

<img width="100px" src="images/laba08_r_a_avg.png">

In [10]:
ru_bar = r.mean(axis=1)
ru_bar.tail()

user_id
939    4.265306
940    3.457944
941    4.045455
942    4.265823
943    3.410714
dtype: float64

In [11]:
ru_bar.shape

(943,)

Для выданного вам пользователя (ID в личном кабинете): 642

In [12]:
a = 642
ra_bar = r.loc[a].mean()
ra_bar

3.6226415094339623

In [13]:
# Review user 642 with his immidate neighbours by index
r.loc[640:650]

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
640,NaN,4.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
642,5.0,4.0,NaN,3.0,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
643,5.0,3.0,NaN,4.0,3.0,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
644,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
645,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
647,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
648,5.0,4.0,NaN,1.0,4.0,NaN,3.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# how many ratings does our user "a" have?
r.loc[a].count()

318

2. Посчитайте меру близости Пирсона выданного вам пользователя со всеми остальными пользователями. Обратите внимание, что корреляция Пирсона считается только на пересечении, то есть вклад вносят только фильмы, оцененные совместно (I<sub>a</sub>, I<sub>u</sub> — множества оцененных пользователями `a` и `u` фильмов). Корреляция с константой (ситуация, когда у пользователя все оценки одинаковые) равна нулю.

Формат: `ID пользователя; корреляция Пирсона`.

<img width="350px" src="images/laba08_pearson.png">

In [17]:
# compute intersection of users with our user "a"
pu = r.mul(r.loc[a], axis=1).div(r.loc[a], axis=1)
pu.loc[640:650]

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
640,NaN,4.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
642,5.0,4.0,NaN,3.0,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
643,5.0,3.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
644,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
645,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
647,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
648,5.0,4.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# compute intersection of user "a" with other users
pa = r.mul(r.loc[a], axis=1).div(r)
pa.loc[640:650]

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
640,NaN,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
642,5.0,4.0,NaN,3.0,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
643,5.0,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
644,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
645,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
647,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
648,5.0,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# compute the the Pearson correlation formula
sau = pu.sub(ru_bar, axis=0) \
     .mul(pa.sub(ra_bar)) \
     .sum(axis=1) \
     .div(pa.sub(ra_bar).pow(2).sum(axis=1).pow(0.5)) \
     .div(pu.sub(ru_bar, axis=0).pow(2).sum(axis=1).pow(0.5))
sau.loc[640:650]

user_id
640    0.269515
641   -0.272530
642    1.000000
643   -0.132156
644    0.455632
645   -0.142107
646    0.171570
647   -0.132093
648    0.199037
649    0.102955
650   -0.026377
dtype: float64

In [21]:
sau.shape

(943,)

3. Посчитайте поправочный коэффициент для корреляции Пирсона на нехватку данных:

<img width="150px" src="images/laba08_reg_coef.png">

In [22]:
# Adjustment coefficient for insufficient number of ratings
adj = pu.count(axis=1).div(50).apply(lambda x: x if x < 1 else 1)
adj.loc[640:650]

user_id
640    0.76
641    0.20
642    1.00
643    1.00
644    0.28
645    0.84
646    0.12
647    0.72
648    1.00
649    0.24
650    1.00
dtype: float64

4. Найдите 30 ближайших пользователей-соседей данного пользователя (`pearson_neighbours`), используя поправленную корреляцию Пирсона: 

<img width="500px" src="images/laba08_pearson_reg_coef.png">

In [24]:
# compute adjusted Pearson correllation
sau_adj = adj * sau
sau_adj.loc[640:650]

user_id
640    0.204832
641   -0.054506
642    1.000000
643   -0.132156
644    0.127577
645   -0.119370
646    0.020588
647   -0.095107
648    0.199037
649    0.024709
650   -0.026377
dtype: float64

In [25]:
# find 30 closest neighbours to our user "a" by adjusted Pearson correlation
pearson_neighbours = sau_adj.sort_values(ascending=False)[1:31]
pearson_neighbours

user_id
921    0.511766
826    0.415054
825    0.385648
721    0.376001
807    0.353948
125    0.349010
577    0.348563
798    0.342644
684    0.338506
224    0.337836
927    0.332941
535    0.325974
398    0.325263
676    0.311132
493    0.304701
454    0.302702
459    0.302659
698    0.302565
671    0.300795
796    0.298483
746    0.298153
545    0.290838
844    0.274008
851    0.273537
87     0.273234
830    0.272543
343    0.270477
620    0.268809
101    0.267453
290    0.263348
dtype: float64

Дальше мы будем использовать только этих соседей для прогноза. Заметим, что строя предсказание для некоторого фильма, мы из фиксированных 30 ближайших пользователей-соседей выбираем только тех, у кого есть оценки для данного фильма, то есть используемых пользователей-соседей для прогноза может быть сильно меньше. Можно находить k ближайших пользователей-соседей, у которых есть оценки по интересующему нас фильму, но это вычислительно гораздо сложнее, поэтому целесообразность этого следует проверять на практике (в рамках данной лабораторной, это лишнее усложнение). 

5. Для всех фильмов найдите прогноз оценки по формуле ниже. Здесь *N(a)* — множество пользователей-соседей, *s(a,u)* — мера близости пользователей из предыдущих пунктов, *|s(a,u)|* — модуль меры близости. 

<img width="250px" src="images/laba08_user_user_cf.png">

In [33]:
# indices of neighbours of user "a"
na = pearson_neighbours.index
na

Int64Index([921, 826, 825, 721, 807, 125, 577, 798, 684, 224, 927, 535, 398,
            676, 493, 454, 459, 698, 671, 796, 746, 545, 844, 851,  87, 830,
            343, 620, 101, 290],
           dtype='int64', name='user_id')

In [47]:
# intersection of neighbour ratings and their similarity metric
sna = r.loc[na].div(r.loc[na]).mul(sau_adj[na], axis=0)
sna

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
921,0.511766,NaN,NaN,NaN,NaN,NaN,NaN,0.511766,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
826,0.415054,0.415054,NaN,0.415054,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
825,NaN,NaN,NaN,NaN,NaN,NaN,0.385648,NaN,0.385648,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
721,0.376001,NaN,NaN,NaN,NaN,NaN,NaN,0.376001,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
807,0.353948,0.353948,NaN,NaN,NaN,NaN,NaN,0.353948,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,0.349010,NaN,NaN,NaN,NaN,NaN,NaN,0.349010,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
577,0.348563,NaN,NaN,0.348563,0.348563,NaN,0.348563,0.348563,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
798,0.342644,0.342644,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
684,0.338506,NaN,NaN,NaN,NaN,NaN,NaN,0.338506,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
# forecast of rating for each movie = weighted average of neighbours ratings
rai_hat = r.loc[na].sub(ru_bar.loc[na], axis=0).mul(sna).sum(axis=0) \
         .div(sna.abs().sum(axis=0)) \
         .add(ra_bar)
rai_hat.head()

item_id
1    4.140585
2    3.634396
3    2.899155
4    4.028789
5    3.319190
dtype: float64

In [49]:
rai_hat.shape

(1682,)

6. Рекомендуйте пользователю 10 фильмов (`pearson_top10`), которые он еще не оценивал, с самыми высокими прогнозами из предыдущего пункта.

In [50]:
pearson_top10 = rai_hat.loc[r.loc[a].isna()].nlargest(10)
pearson_top10

item_id
160     5.163311
347     5.163311
915     5.163311
1143    5.163311
990     5.086927
1065    5.086927
562     5.041996
344     5.038226
982     5.008848
1244    5.008848
dtype: float64

7. Сравните результат с топ-10 фильмами, полученными по нижней границе доверительного интервала Wilson - поле `top10_lower` из Лабы 6S. Найдите количество общих фильмов в этих двух десятках.

#### top10 films by the lower border of confidence interval from lab06
[64, 479, 318, 98, 483, 603, 427, 114, 408, 50]

### Save the results to json file

In [51]:
# Make a dictionary to present the results in the required format
d = {
    "average_film_ratings": average_film_ratings,
    "average_user_ratings": average_user_ratings,
    "completeness": completeness,
    "pearson_neighbours": [item for item in pearson_neighbours.index],
    "pearson_top10": [item for item in pearson_top10.index]
}
d

{'average_film_ratings': 59.453,
 'average_user_ratings': 106.0445,
 'completeness': 0.063,
 'pearson_neighbours': [921,
  826,
  825,
  721,
  807,
  125,
  577,
  798,
  684,
  224,
  927,
  535,
  398,
  676,
  493,
  454,
  459,
  698,
  671,
  796,
  746,
  545,
  844,
  851,
  87,
  830,
  343,
  620,
  101,
  290],
 'pearson_top10': [160, 347, 915, 1143, 990, 1065, 562, 344, 982, 1244]}

In [52]:
# Preview json string
json.dumps(d)

'{"average_film_ratings": 59.453, "average_user_ratings": 106.0445, "completeness": 0.063, "pearson_neighbours": [921, 826, 825, 721, 807, 125, 577, 798, 684, 224, 927, 535, 398, 676, 493, 454, 459, 698, 671, 796, 746, 545, 844, 851, 87, 830, 343, 620, 101, 290], "pearson_top10": [160, 347, 915, 1143, 990, 1065, 562, 344, 982, 1244]}'

In [53]:
# Save the dictionary to a json file
with open('../../lab08.json', 'w') as f:
    json.dump(d, f)

## ✨ Лаба 8. Суперачивка. Рекомендательная система фильмов MovieLens — 2

### Дедлайн

⏰ Четверг, 06 июня 2019 года, 23:59.

### Задача

По имеющимся данным о рейтингах фильмов (MovieLens: 100 000 рейтингов) построить коллаборативную рекомендательную систему. Вторая часть (ачивка) — это продолжение работы, начатой в [Лабе 8](lab08.md). Мы предлагаем построить новую систему на основе коллаборативной фильтрации путём подсчёта базовых предикторов и создания с ними Item-Item Recommender System.

#### Обработка данных на вход

Входные данные: датасет MovieLens и `id` выданного вам пользователя для расчёта — тот же, что для [Лабы 8](lab08.md).

#### Обработка данных на выход

**Часть 3. Базовые предикторы:**

1. Глобальное среднее 𝞵 (`average_rating`) по всему датасету. `Сумма всех оценок по всем фильмам / Количество всех оценок по всем фильмам.`

2. Базовый предиктор для каждого пользователя (суммирование по фильмам, оцененным данным пользователем). Здесь I<sub>a</sub> — множество фильмов, по которым у пользователя есть рейтинги, а |I<sub>a</sub>| — их количество.

<img width="200px" src="images/laba08s_base_u.png">

3. Базовый предиктор для каждого фильма (суммирование по пользователям, поставившим оценку данному фильму). Здесь U<sub>i</sub> — множество пользователей, которые оценили данный фильм, а |U<sub>i</sub>| — их количество.

<img width="250px" src="images/laba08s_base_i.png">

4. Базовый предиктор для каждого пользователя и каждого фильма:

<img width="140px" src="images/laba08s_base_ui.png">

**Часть 4. Item-item CF:**

1. Вычесть из всех рейтингов r<sub>ui</sub> базовый предиктор b<sub>ui</sub> из пункта 3.4 (для всей таблицы рейтингов). Если рейтинга нет, то можно поставить результат = 0.
2. Найдите попарные меры близости (косинус) для всех фильмов, используя очищенные оценки из пункта 4.1. Суммирование идет по всем пользователям.

<img width="350px" src="images/laba08s_cosine_items.png">

3. Для каждого фильма, по которому у данного пользователя не стоит рейтинг, найдите:

   * [a] 30 ближайших фильмов-соседей для этого фильма (среди всех фильмов, а не фильмов, оценённых пользователем).

   * [b] прогноз оценки пользователя по формуле (базовый предиктор из пункта 3.4).  Здесь *S(i)*- множество фильмов-соседей для фильма *i*, по которым у данного пользователя есть оценка.
<img width="300px" src="images/laba08s_item_item_cf.png">

     Заметим, что суммирование идет только по тем фильмам-соседям, которые оценил пользователь.

4. Рекомендуйте пользователю 10 фильмов (`predicators_top10`) с самыми высокими оценками из пункта 4.3.

5. При подсчете прогноза по формуле из пункта 4.3 отфильтруйте всех соседей с отрицательной близостью.

6. Рекомендуйте пользователю 10 фильмов (`predicators_positive_top10`) с самыми высокими оценками из пункта 4.5.

Задание для себя: посмотрите совпадают ли топы в пунктах 4 и 6. Найдите количество общих фильмов в топ-10 из пункта 2.6 и топ-10 из пункта 4.6.

#### Проверка

Результат следует сохранить в файл lab08s.json в своей домашней директории в следующем формате:

```
{
    "average_rating": 3.1111,
    "predicators_positive_top10": [489, 604, 1449, 611, 656, 170, 847, 793, 615, 963],
    "predicators_top10": [123, 604, 124, 456, 656, 170, 847, 793, 615, 963]
}
```

При равенстве соответствующих метрик сортировать необходимо по возрастанию `id`, рассматривать как число.

Округление метрик — до 4 знаков после запятой.

Для получения `True` по двум последним ключам **точность совпадения должна быть не менее 0.8**.

Проверка осуществляется [автоматическим скриптом](http://lk.newprolab.com/lab/laba08s) из Личного кабинета.

## Решение

**Часть 3. Базовые предикторы:**

1. Глобальное среднее 𝞵 (`average_rating`) по всему датасету. `Сумма всех оценок по всем фильмам / Количество всех оценок по всем фильмам.`

In [151]:
mu = r.sum().sum() / r.count().sum()
mu

3.52986

2. Базовый предиктор для каждого пользователя (суммирование по фильмам, оцененным данным пользователем). Здесь I<sub>a</sub> — множество фильмов, по которым у пользователя есть рейтинги, а |I<sub>a</sub>| — их количество.

<img width="200px" src="images/laba08s_base_u.png">

In [152]:
bu = r.sub(mu).sum(axis=1).div(r.count(axis=1).add(10), axis=0)
bu.loc[640:650]

user_id
640    0.640716
641    0.476112
642    0.089953
643    0.198374
644    0.379728
645    0.396645
646   -0.461139
647    0.253943
648   -0.261786
649    0.184805
650   -0.363821
dtype: float64

3. Базовый предиктор для каждого фильма (суммирование по пользователям, поставившим оценку данному фильму). Здесь U<sub>i</sub> — множество пользователей, которые оценили данный фильм, а |U<sub>i</sub>| — их количество.

<img width="250px" src="images/laba08s_base_i.png">

In [153]:
bi = r.sub(bu, axis=0).sub(mu).sum(axis=0).div(r.count(axis=0).add(25))
bi.head(10)

item_id
1     0.292925
2    -0.213032
3    -0.320284
4    -0.019912
5    -0.158643
6     0.057544
7     0.233084
8     0.344186
9     0.299534
10    0.208049
dtype: float64

4. Базовый предиктор для каждого пользователя и каждого фильма:

<img width="140px" src="images/laba08s_base_ui.png">

In [154]:
b = pd.DataFrame(data=mu, index=bu.index, columns=bi.index).add(bu, axis=0).add(bi, axis=1)
b.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,3.900367,3.394410,3.287158,3.587530,3.448799,3.664986,3.840526,3.951628,3.906975,3.815490,...,3.565444,3.603299,3.589651,3.551190,3.598750,3.524503,3.601427,3.562965,3.607618,3.593175
2,3.977628,3.471671,3.364418,3.664791,3.526060,3.742247,3.917787,4.028889,3.984236,3.892751,...,3.642705,3.680560,3.666912,3.628451,3.676011,3.601764,3.678687,3.640226,3.684879,3.670436
3,3.203841,2.697884,2.590631,2.891004,2.752273,2.968460,3.143999,3.255101,3.210449,3.118964,...,2.868917,2.906772,2.893125,2.854664,2.902224,2.827977,2.904900,2.866439,2.911092,2.896649
4,4.389943,3.883986,3.776733,4.077106,3.938375,4.154562,4.330101,4.441204,4.396551,4.305066,...,4.055020,4.092874,4.079227,4.040766,4.088326,4.014079,4.091002,4.052541,4.097194,4.082751
5,3.202647,2.696690,2.589438,2.889810,2.751079,2.967266,3.142806,3.253908,3.209256,3.117771,...,2.867724,2.905579,2.891932,2.853470,2.901030,2.826784,2.903707,2.865245,2.909898,2.895455


In [155]:
b.shape

(943, 1682)

**Часть 4. Item-item CF:**

1. Вычесть из всех рейтингов r<sub>ui</sub> базовый предиктор b<sub>ui</sub> из пункта 3.4 (для всей таблицы рейтингов). Если рейтинга нет, то можно поставить результат = 0.

In [156]:
rlessb = r.sub(b).fillna(0)
rlessb.loc[640:650]

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
640,0.000000,0.042455,0.0,-0.150665,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
641,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
642,1.087262,0.593219,0.0,-0.599901,0.000000,0.0,0.000000,1.036001,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
643,0.978841,-0.515203,0.0,0.291678,-0.569591,0.0,0.038682,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
644,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
645,0.000000,0.000000,0.0,0.093407,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
646,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
647,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
648,1.439001,0.944958,0.0,-2.248162,0.890569,0.0,-0.501157,0.000000,-2.567607,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


2. Найдите попарные меры близости (косинус) для всех фильмов, используя очищенные оценки из пункта 4.1. Суммирование идет по всем пользователям.

<img width="350px" src="images/laba08s_cosine_items.png">

In [157]:
sij = rlessb.T.dot(rlessb).div(rlessb.pow(2).sum(axis=0).pow(0.5), axis=0).div(rlessb.pow(2).sum(axis=0).pow(0.5), axis=1)
sij.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
item_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,-0.003603,-0.012158,-0.035144,0.039609,0.022780,0.021228,0.052695,-0.004018,-0.015940,...,0.077118,0.0,0.00000,0.00000,0.028896,0.0,0.0,0.0,0.039629,-0.018718
2,-0.003603,1.000000,-0.001984,0.050458,0.002228,-0.003639,-0.016975,0.040146,-0.089526,-0.007177,...,0.000000,0.0,0.00000,0.00000,0.000000,0.0,0.0,0.0,0.022621,0.016415
3,-0.012158,-0.001984,1.000000,-0.113536,-0.005417,0.060713,-0.024408,-0.094746,-0.016745,-0.039964,...,0.000000,0.0,0.00000,0.00000,0.183583,0.0,0.0,0.0,0.000000,0.004880
4,-0.035144,0.050458,-0.113536,1.000000,-0.149507,-0.019089,0.005145,0.110511,0.033087,0.010968,...,0.000000,0.0,-0.11599,-0.11599,0.089827,0.0,0.0,0.0,0.001824,-0.048334
5,0.039609,0.002228,-0.005417,-0.149507,1.000000,-0.031989,-0.015494,0.038203,-0.042447,-0.036512,...,0.000000,0.0,0.00000,0.00000,0.000000,0.0,0.0,0.0,0.000000,0.027241


3. Для каждого фильма, по которому у данного пользователя не стоит рейтинг, найдите:

   * [a] 30 ближайших фильмов-соседей для этого фильма (среди всех фильмов, а не фильмов, оценённых пользователем).

   

In [158]:
# найдем 30 ближайших соседей для каждого фильма, по которому у пользователя "а" не стоит рейтинг
%time s30 = sij.apply(lambda x: [v for v in x.sort_values(ascending=False)[1:31].index.astype('int64')])
s30.head()

CPU times: user 1.12 s, sys: 4 ms, total: 1.13 s
Wall time: 1.12 s


item_id
1    [588, 933, 174, 870, 928, 426, 1117, 845, 95, ...
2    [385, 566, 1419, 54, 233, 926, 1555, 808, 405,...
3    [1230, 409, 1677, 282, 1231, 33, 1552, 783, 16...
4    [85, 414, 1554, 1626, 144, 1203, 1550, 12, 152...
5    [1554, 581, 376, 1550, 397, 1053, 857, 830, 85...
dtype: object

In [172]:
# Convert 30 neighbours indices into dictionary for further reference
d30 = dict(s30)

In [160]:
# сделаем матрицу cos только для фильмов соседей
# сначала инициализируем пустую матрицу
sn = pd.DataFrame(index=sij.index, columns=sij.columns)
sn.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
item_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [161]:
# заполним ее косинусами соседей фильмов в индексе
for i in range(1, 1683):
    for j in d30[i]:
        sn.at[i,j] = sij.at[i,j]
sn.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
item_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.183583,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


   * [b] прогноз оценки пользователя по формуле (базовый предиктор из пункта 3.4).  Здесь *S(i)*- множество фильмов-соседей для фильма *i*, по которым у данного пользователя есть оценка.
<img width="300px" src="images/laba08s_item_item_cf.png">

     Заметим, что суммирование идет только по тем фильмам-соседям, которые оценил пользователь.

In [162]:
# множество фильмов соседей, по которым у пользователя "а" есть оценка
sna = sn.mul(r.loc[a], axis=1).div(r.loc[a], axis=1)
sna.head(10)

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
item_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,0.110511,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
# прогноз оценки пользователя по формуле
ra_hat = sna.mul(rlessb.loc[a], axis=1).sum(axis=1).div(sna.abs().sum(axis=1), axis=0).add(b.loc[a], axis=0)
ra_hat[r.loc[a].isna()].head(10)

item_id
3     3.862711
5     2.634702
6          NaN
7     3.484790
9     3.858685
10    4.050063
11    3.527963
12    3.646487
14    3.776998
16    3.782140
dtype: float64

4. Рекомендуйте пользователю 10 фильмов (`predicators_top10`) с самыми высокими оценками из пункта 4.3.

In [164]:
predicators_top10 = ra_hat[r.loc[a].isna()].nlargest(10)
predicators_top10

item_id
558     5.443018
744     5.260848
902     5.194650
1625    5.175012
923     5.134271
1237    5.126466
309     5.104256
246     5.087646
1512    5.058439
45      5.057613
dtype: float64

5. При подсчете прогноза по формуле из пункта 4.3 отфильтруйте всех соседей с отрицательной близостью.

In [165]:
snp = sna.applymap(lambda x: 0 if x < 0 else x)
snp.head(10)

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
item_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,0.110511,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [166]:
# обновим прогноз с соседями только с положительной близостью по косинусу
ra_hat_p = snp.mul(rlessb.loc[a], axis=1).sum(axis=1).div(snp.abs().sum(axis=1), axis=0).add(b.loc[a], axis=0)
ra_hat_p[r.loc[a].isna()].head(10)

item_id
3     3.862711
5     2.634702
6          NaN
7     3.484790
9     3.858685
10    4.050063
11    3.527963
12    3.646487
14    3.776998
16    3.782140
dtype: float64

6. Рекомендуйте пользователю 10 фильмов (`predicators_positive_top10`) с самыми высокими оценками из пункта 4.5.

In [167]:
predicators_positive_top10 = ra_hat_p[r.loc[a].isna()].nlargest(10)
predicators_positive_top10

item_id
558     5.443018
744     5.260848
902     5.194650
1625    5.175012
923     5.134271
1237    5.126466
309     5.104256
246     5.087646
1512    5.058439
45      5.057613
dtype: float64

Задание для себя: посмотрите совпадают ли топы в пунктах 4 и 6. Найдите количество общих фильмов в топ-10 из пункта 2.6 и топ-10 из пункта 4.6.

* __6:__ 558 744 902 1625 923 1237 309 246 1512 45
* __4:__ 558 744 902 1625 923 1237 309 246 1512 45

### Save the results to json file

In [168]:
# Make a dictionary to present the results in the required format
ds = {
    "average_rating": mu,
    "predicators_positive_top10": [item for item in predicators_positive_top10.index],
    "predicators_top10": [item for item in predicators_top10.index]
}
ds

{'average_rating': 3.52986,
 'predicators_positive_top10': [558,
  744,
  902,
  1625,
  923,
  1237,
  309,
  246,
  1512,
  45],
 'predicators_top10': [558, 744, 902, 1625, 923, 1237, 309, 246, 1512, 45]}

In [169]:
# Preview json string
json.dumps(ds)

'{"average_rating": 3.52986, "predicators_positive_top10": [558, 744, 902, 1625, 923, 1237, 309, 246, 1512, 45], "predicators_top10": [558, 744, 902, 1625, 923, 1237, 309, 246, 1512, 45]}'

In [170]:
# Save the dictionary to a json file
with open('../../lab08s.json', 'w') as f:
    json.dump(ds, f)